In [ ]:
# process
# load excel sheets with data
# set parameters (time of bins, frequency bands, designate one as 'tremor frequency')
# calculate averages for each animal
# compile averages for each animal into sheet
# calculate TPR for each bin 
# output data in exact format for prism
    # parameterize group and animal numbers as input
    # organize data as columns == animals, rows == time bins, columns organized by group

In [1]:
import pandas as pd
import numpy as np

In [2]:
fn = "/Users/rebeccakrall/Data/Tremor Project/TPR data all animals.xlsx"
excel_tables = pd.read_excel(fn, sheet_name=None)


In [3]:
animal_key = pd.read_excel("/Users/rebeccakrall/Data/Tremor Project/Animal_group_key.xlsx", index_col = 0)

In [4]:
animals = [a for a in excel_tables.keys() if a != 'Summary']

In [5]:
sheet = excel_tables[animals[0]]

In [6]:
data = sheet.iloc[0:256, :].set_index('f (Hz)').iloc[:, 2:]
new_cols = np.linspace(0, 10.24 * (sheet.shape[1]-1), sheet.shape[1])/60

data.columns = new_cols[2:-1]

In [7]:
parameters = {'hz': [2,8,14,25], 'bin_time': 5, 'col_time':10.24, 'exp_length' : 60, 'tremor_freq': 1}

In [8]:
def process_excel_sheet(sheet, parameters):
    
    data = sheet.iloc[0:256, :].set_index('f (Hz)').iloc[:, 2:]
    new_cols = np.linspace(0, parameters['col_time'] * (sheet.shape[1]-1), sheet.shape[1])/60
    data.columns = new_cols[2:-1]

    col_bins = np.linspace(0, parameters['exp_length'] , int(parameters['exp_length'] / parameters['bin_time']) +1)
    frequency_bins  = pd.cut(data.index, bins = parameters['hz'])
    time_bins = pd.cut(data.columns, bins = col_bins) 
    a = data.groupby([frequency_bins], observed = False).mean().T.groupby([time_bins], observed = False).mean() * 1000

    non_target_columns = [col for col in range(len(a.columns)) if col != parameters['tremor_freq']]
    tpr = 2 * a.iloc[:, parameters['tremor_freq']] / a.iloc[:, non_target_columns].sum(axis=1)
    # tpr = 2 * a.iloc[:,parameters['tremor_freq']] / (a.iloc[:,0]+ a.iloc[:,2])

    return a, tpr
   

In [9]:
def compile_excel_sheets(excel_tables, animals, animal_key, parameters):
    columns = pd.MultiIndex(levels=[[], []], codes=[[], []], names=[ 'Group', 'Subject'])
    tprs = pd.DataFrame(columns = columns)

    columns2 = pd.MultiIndex(levels=[[], [], []], codes=[[], [],[]], names=[ 'Group', 'Subject', 'Frequency'])
    freqs = pd.DataFrame(columns=columns2)

    all_freq = []
    for an in animals:
        if int(an) in animal_key.index:
            group = animal_key.loc[int(an), 'GROUP']
            column_index = (group, an)

            freq, trp = process_excel_sheet(excel_tables[an], parameters)
            
            multi_index = pd.MultiIndex.from_arrays([[group] * 3, [an] * 3, freq.columns], names=['Group', 'Subject', 'Frequency'])
            freq.columns = multi_index
            # freqs[multi_index] = freq

            all_freq.append(freq)
            tprs[column_index] = trp

    freqs = pd.concat(all_freq, axis = 1)

    return freqs, tprs

In [51]:
non_target_columns = [col for col in range(len(a.columns)) if col != parameters['tremor_freq']]

In [56]:
a, tpr = process_excel_sheet(excel_tables['11'], parameters)

In [11]:
freqs, tprs = compile_excel_sheets(excel_tables, animals, animal_key, parameters)

In [15]:
freqs = freqs.sort_index(axis = 1)

In [16]:
freqs

Group                 1                                                \
Subject              11                                12               
Frequency        (2, 8]    (8, 14]   (14, 25]      (2, 8]     (8, 14]   
(0.0, 5.0]    194.24824  71.090213  14.772575  220.582192  115.534452   
(5.0, 10.0]   67.937679  30.983979   6.499227  112.254478   52.468522   
(10.0, 15.0]  11.835019  22.129304   3.523532   70.321174   65.949985   
(15.0, 20.0]   0.538345   1.112722   0.634037   29.314684   22.865159   
(20.0, 25.0]  74.081411  41.556834   7.778708   19.838681    24.22087   
(25.0, 30.0]  91.917646    52.9152   9.977079   18.783586   19.385207   
(30.0, 35.0]   7.083173   8.373174   3.407546    11.32162    7.668791   
(35.0, 40.0]  15.959205  19.969055   4.330484    5.200733     9.16984   
(40.0, 45.0]   0.133972   0.626267   0.554065    0.719583      0.6507   
(45.0, 50.0]   0.283087   0.904476   0.776319    1.846231    1.643655   
(50.0, 55.0]   7.577401  11.400451   5.788751    3.811658    2.573174   
(55.0, 60.0]   0.236756   0.284824    0.67801    0.294343    0.267935   

Group                                                                 ...  \
Subject                          13                               14  ...   
Frequency      (14, 25]      (2, 8]    (8, 14]  (14, 25]      (2, 8]  ...   
(0.0, 5.0]     21.46117  148.285364  69.571485   10.6045  184.366947  ...   
(5.0, 10.0]   12.578258   80.657544  33.580975  5.788374   89.514294  ...   
(10.0, 15.0]  12.345032    6.478613   5.956072  0.790369   35.130465  ...   
(15.0, 20.0]   8.620252    0.472051   0.608966   0.42485   29.683773  ...   
(20.0, 25.0]   6.978707    1.167316   1.219799  1.078637    7.630664  ...   
(25.0, 30.0]    7.68376   28.620368  36.420948  6.759397    0.241781  ...   
(30.0, 35.0]   4.424132    3.729538   3.110197  1.064376    0.542964  ...   
(35.0, 40.0]   3.104008    0.038237   0.060305  0.149738    0.276172  ...   
(40.0, 45.0]   0.516032    0.312966   0.732463  2.246863    0.490695  ...   
(45.0, 50.0]   1.097844     0.32323   0.979677  0.968424    0.108453  ...   
(50.0, 55.0]   1.222585    0.764748   2.960597  2.645743    0.129822  ...   
(55.0, 60.0]   0.536569    2.396395  15.537454  8.174143    0.154637  ...   

Group                  6                                                   \
Subject               77           78                                  79   
Frequency       (14, 25]       (2, 8]     (8, 14]    (14, 25]      (2, 8]   
(0.0, 5.0]     22.635608    69.956279   44.117612    5.464217  104.513146   
(5.0, 10.0]    30.043803    16.096363   76.626855    4.627165   10.434177   
(10.0, 15.0]  108.935328    10.315196   50.510854     3.36837   29.870295   
(15.0, 20.0]   20.291206     2.522165    7.738792    1.420481    38.41016   
(20.0, 25.0]   10.586393     2.624707    9.862966    1.467409    3.753953   
(25.0, 30.0]    8.326748     1.454905    7.101244    0.961469      4.4646   
(30.0, 35.0]    7.727561     0.733915    2.149497    0.864026    7.012949   
(35.0, 40.0]    6.354349        33.16  163.071766   10.311386    6.572776   
(40.0, 45.0]    7.933523    25.725213  117.903004    9.750712    4.363199   
(45.0, 50.0]    7.253101  2062.179183   707.91809  172.105082    3.629098   
(50.0, 55.0]    8.126663    21.510234  114.782734   11.212879    2.399406   
(55.0, 60.0]    6.073151    34.375886  152.506468    9.955615    1.637838   

Group                                                                 
Subject                                     80                        
Frequency        (8, 14]   (14, 25]     (2, 8]     (8, 14]  (14, 25]  
(0.0, 5.0]     119.15035  14.542268  65.880383   99.752744   9.37388  
(5.0, 10.0]   105.556961    4.97808   16.37043   81.802781  5.490898  
(10.0, 15.0]  270.517517  23.066946   7.825968   54.418928  2.794572  
(15.0, 20.0]  209.052633  41.725779   4.247217   13.137058  1.355068  
(20.0, 25.0]   27.690512   3.510533   1.148562   11.226778  1.157787  
(25.0, 30.0]   33.246894